In [1]:
import os
from dataclasses import dataclass
from pathlib import Path
from ImageClassifier.constants import *
from ImageClassifier.utils.common import read_yaml, create_directories
import urllib.request as request
import zipfile
from ImageClassifier import logger
from ImageClassifier.utils.common import get_size

In [2]:
current_path = os.getcwd()
while not current_path.endswith('Image_classification_E2E'):
    print(current_path)
    os.chdir('../')
    current_path = os.getcwd()
print(os.getcwd())

/Users/abdellah.kaissari/Desktop/Abdellah_Perso/Image_classification_E2E/research
/Users/abdellah.kaissari/Desktop/Abdellah_Perso/Image_classification_E2E


In [3]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
class ConfigurationManager:
    def __init__(
            self, 
            config_filepath: Path = CONFIG_FILE_PATH, 
            params_filepath: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config


In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, 
                filename=self.config.local_data_file
            )
            logger.info(f'{filename} downloaded with following headers: {headers}')
        else:
            logger.info(f'File alreadu exists of size {get_size(Path(self.config.local_data_file))}')

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)        

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-20 08:26:42,642: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-06-20 08:26:42,643: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-20 08:26:42,643: INFO: common: created directory at: artifacts]
[2024-06-20 08:26:42,643: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-20 08:26:43,309: INFO: 220288106: artifacts/data_ingestion/data.zip downloaded with following headers: Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7099:1CAFA8:1A1967C:1B67270:6673C869
Accept-Ranges: bytes
Date: Thu, 20 Jun 2024 06:26:43 GMT
Via: 1.1 varnish
X-Served-By: cache-par-lfpg1960081-PA